# **保存和载入Keras模型**

### **介绍**
Keras模型由许多组件构成：

+ 模型结构或配置，它指定模型包含哪些层以及如何连接它们。
+ 一组权重值（即“模型的状态”）
+ 优化器（在编译模型时定义）
+ 一组损失和指标（在编译模型时定义或者调用`add_loss()`或`add_metric()`）

通过Keras API，可以将这些片段一次性保存到磁盘，或者仅选择性地保存其中一部分：

+ 标准的做法是，将模型的所有内容以TensorFlow SavedModel格式（或更旧的Keras H5格式）保存到单个文件中。
+ 仅保存模型的架构/配置，通常保存为JSON文件。
+ 仅保存模型的权重值，通常在训练模型时使用。

接下来让我们来讨论上述方法的使用：什么时候使用它们？它们如何运作的？

### 保存和加载快捷方式

如果你只有10秒钟的时间阅读本指南，则需要了解以下内容。

**保存Keras模型：**

In [ ]:
model = ...  # 创建一个模型 (Sequential模型, 函数式模型, 或者子类化模型)
model.save('path/to/location')

**载入模型：**

In [ ]:
from tensorflow import keras
model = keras.models.load_model('path/to/location')

接下来，让我们来讨论具体细节。

### **引入**

In [1]:
import numpy as np
import tensorflow as tf
from tensorflow import keras

### **整个模型的保存和载入**

你可以将整个模型保存到单个文件中。它将包括：
+ 模型的架构/配置
+ 模型的权重值（在训练过程中学习的）
+ 模型被调用的编译信息（如果调用了`compile()`）
+ 优化器及其状态（这使你可以在中断的位置重新开始训练）

**API：**
+ `model.save()`或[`tf.keras.models.save_model()`](https://www.tensorflow.org/api_docs/python/tf/keras/models/save_model)
+ [`tf.keras.models.load_model()`](https://www.tensorflow.org/api_docs/python/tf/keras/models/load_model)

你可以使用两种格式将整个模型保存到磁盘：**TensorFlow SavedModel格式**和**较老的Keras H5格式**。推荐的格式为SavedModel。使用`model.save()`时是默认设置。

您可以通过以下方式切换到H5格式：
+ 将`save_format='h5'`传递给`save()`
+ 将传递给`save()`的文件名的末尾加上`.h5`或者`.keras`

#### SavedModel格式
举例

In [2]:
def get_model():
    # 创建一个简单模型
    inputs = keras.Input(shape=(32,))
    outputs = keras.layers.Dense(1)(inputs)
    model = keras.Model(inputs, outputs)
    model.compile(optimizer="adam", loss="mean_squared_error")
    return model


model = get_model()

# 训练模型
test_input = np.random.random((128, 32))
test_target = np.random.random((128, 1))
model.fit(test_input, test_target)

# 调用`save('my_model')`创建一个SavedModel，文件夹命名为`my_model`.
model.save("my_model")

# 非常方便的重新载入
reconstructed_model = keras.models.load_model("my_model")

# 验证一下
np.testing.assert_allclose(
    model.predict(test_input), reconstructed_model.predict(test_input)
)

# 重建的模型通过编译，由于保留了优化器状态，因此可以恢复训练：
reconstructed_model.fit(test_input, test_target)

4/4 [==============================] - 0s 3ms/step - loss: 0.4323
Instructions for updating:
This property should not be used in TensorFlow 2.0, as updates are applied automatically.
Instructions for updating:
This property should not be used in TensorFlow 2.0, as updates are applied automatically.
INFO:tensorflow:Assets written to: my_model/assets
4/4 [==============================] - 0s 2ms/step - loss: 0.3800


**SavedModel包含了什么**

调用`model.save('my_model')`创建一个名为`my_model`的文件夹，其中包含以下内容：

In [3]:
ls my_model

assets/  saved_model.pb  variables/


模型结构和训练配置（包括优化器，损失和指标）存储在`save_model.pb`中。权重保存在`variables/`目录中。

有关SavedModel格式的详细信息，请参见[SavedModel指南（磁盘上的SavedModel格式）](https://www.tensorflow.org/guide/saved_model#the_savedmodel_format_on_disk)。

#### **SavedModel如何处理自定义对象**

保存模型及其层时，SavedModel格式存储类名称，**调用函数**，损失和权重（以及配置）。调用函数定义了模型/层的计算图。

在没有模型/层配置的情况下，调用函数将创建一个类似原始模型的模型，该模型可以进行训练，评估并用于预测。

尽管如此，在编写自定义模型或层的类时，定义`get_config`和`from_config`方法始终是一个好习惯。这样，你便可以在需要时轻松地更新计算。有关更多信息，请参见关于[自定义对象](https://www.tensorflow.org/guide/keras/save_and_serialize#custom_objects)的部分。

以下是从SavedModel格式加载自定义层而**不覆盖**`config`方法时，操作示例。

In [4]:
class CustomModel(keras.Model):
    def __init__(self, hidden_units):
        super(CustomModel, self).__init__()
        self.dense_layers = [keras.layers.Dense(u) for u in hidden_units]

    def call(self, inputs):
        x = inputs
        for layer in self.dense_layers:
            x = layer(x)
        return x


model = CustomModel([16, 16, 10])
# 通过调用它创建模型
input_arr = tf.random.uniform((1, 5))
outputs = model(input_arr)
model.save("my_model")

# 删除自定义模型类，以确保加载程序无法访问它。
del CustomModel

loaded = keras.models.load_model("my_model")
np.testing.assert_allclose(loaded(input_arr), outputs)

print("Original model:", model)
print("Loaded model:", loaded)

INFO:tensorflow:Assets written to: my_model/assets
Original model: <__main__.CustomModel object at 0x7f56ee89c668>
Loaded model: <tensorflow.python.keras.saving.saved_model.load.CustomModel object at 0x7f56ee8950f0>


如上面的示例所示，加载程序动态创建一个行为类似于原始模型的新模型类。

### **Keras H5格式**

Keras还支持保存包含模型的架构，权重值和`compile()`信息的单个HDF5文件。它是SavedModel的轻量替代方案。

**示例：**

In [5]:
model = get_model()

# 训练模型
test_input = np.random.random((128, 32))
test_target = np.random.random((128, 1))
model.fit(test_input, test_target)

# 调用`save('my_model.h5')`创建一个名为`my_model.h5`的h5文件
model.save("my_h5_model.h5")

# 非常方便的重新载入
reconstructed_model = keras.models.load_model("my_h5_model.h5")

# 验证一下：
np.testing.assert_allclose(
    model.predict(test_input), reconstructed_model.predict(test_input)
)

# 重建的模型通过编译，由于保留了优化器状态，因此可以恢复训练：
reconstructed_model.fit(test_input, test_target)

4/4 [==============================] - 0s 2ms/step - loss: 2.0320


#### **局限性**
与SavedModel格式相比，H5文件中没有包含如下两点：
+ 通过`model.add_loss()`和`model.add_metric()`添加的外部损失和指标不会被保存（与SavedModel不同）。如果你的模型上有这样的损失和指标，并且想要恢复训练，则需要在加载模型后重新添加这些损失。请注意，通过`self.add_loss()`和`self.add_metric()`在层内部创建的损耗/指标是会被保存的。只要该层被加载，这些损耗和度量就被保留，因为它们是该层的调用方法的一部分。
+ 自定义对象（如自定义层）的计算图不包含在保存的文件中。在加载时，Keras将需要访问这些对象的Python类/函数以重建模型。请参阅[自定义对象](https://www.tensorflow.org/guide/keras/save_and_serialize#custom_objects)。

### **保存模型架构**

模型的配置（或架构）指定了模型包含的层以及如何连接这些层*。如果你有模型的配置，则可以使用新的初始化状态，配合权重创建模型，而无需编译信息。

*请注意，这仅适用于使用函数式API或`Sequential`API定义的模型，而非子类化模型。

### Sequential模型或函数式API模型的配置
这两个类型的模型是显式的层：它们的配置始终以结构化形式呈现和使用。

**APIs**
+ `get_config()`和`from_config()`
+ `tf.keras.models.model_to_json()`和[`tf.keras.models.model_from_json()`](https://www.tensorflow.org/api_docs/python/tf/keras/models/model_from_json)

**`get_config()`和`from_config()`**

调用`config = model.get_config()`将返回一个包含模型配置的Python字典。然后可以通过`Sequential.from_config(config)`（对于Sequential模型）或`Model.from_config(config)`（对于函数式API模型）来重建相同的模型。

相同的工作流程也适用于任何可序列化层。

**层的例子：**

In [6]:
layer = keras.layers.Dense(3, activation="relu")
layer_config = layer.get_config()
new_layer = keras.layers.Dense.from_config(layer_config)

**Sequential模型的例子：**

In [7]:
model = keras.Sequential([keras.Input((32,)), keras.layers.Dense(1)])
config = model.get_config()
new_model = keras.Sequential.from_config(config)

**函数式API模型的例子：**

In [8]:
inputs = keras.Input((32,))
outputs = keras.layers.Dense(1)(inputs)
model = keras.Model(inputs, outputs)
config = model.get_config()
new_model = keras.Model.from_config(config)

#### **to_json()和[tf.keras.models.model_from_json()](https://www.tensorflow.org/api_docs/python/tf/keras/models/model_from_json)**

这类似于`get_config` / `from_config`，不同之处在于它将模型转换为JSON字符串，然后可以在没有原始模型类的情况下进行加载。它只能应用于模型，并不适用于层。

**例子：**

In [9]:
model = keras.Sequential([keras.Input((32,)), keras.layers.Dense(1)])
json_config = model.to_json()
new_model = keras.models.model_from_json(json_config)

### **自定义对象**

**自定义模型和层**

子类化模型和层的架构在`__init__`和`call`方法中进行定义。它们被定义为Python字节码，无法序列化为与JSON兼容的配置--你可以尝试序列化字节码（例如通过pickle），但这完全是不安全的，这意味着你的模型无法加载到其他系统上。

为了保存/加载具有自定义层的模型或子类化模型，你应该覆盖`get_config`和选择性的覆盖`from_config`方法。另外，你应该注册自定义对象，以便Keras能识别它。

**自定义方法**

自定义方法（例如，激活损失或初始化）不需要`get_config`方法。只要将方法名称注册为自定义对象，就足以通过该方法名称进行加载。

**仅加载TensorFlow图**

可以加载Keras生成的TensorFlow图。如果这样做，则无需提供任何`custom_object`。你可以这样做：

In [10]:
model.save("my_model")
tensorflow_graph = tf.saved_model.load("my_model")
x = np.random.uniform(size=(4, 32)).astype(np.float32)
predicted = tensorflow_graph(x).numpy()

INFO:tensorflow:Assets written to: my_model/assets


请注意，此方法有几个缺点：
+ 出于可追溯性的原因，你应该始终可以访问所使用的自定义对象，毕竟你肯定不想将无法重新创建的模型投入生产。
+ `tf.saved_model.load`返回的对象不是Keras模型。因此，它并不是那么方便使用的。例如，你将无权访问`.predict()`或`.fit()`

即使不鼓励使用它，它也可以在必要时为你提供帮助，例如，如果你丢失了自定义对象的代码或在使用`tf.keras.models.load_model()`加载模型时遇到问题时使用。

你可以在[`tf.saved_model.load`页面](https://www.tensorflow.org/api_docs/python/tf/saved_model/load)找到有关它的更多信息。

#### 定义配置方法
规范：
+ 为了保证Keras中，模型架构和保存API的兼容，`get_config`应该返回一个JSON可序列化的字典。
+ `from_config(config)`(`classmethod`)应该返回通过配置创建的新层或模型对象。默认实现返回`cls(** config)`。

**示例：**

In [11]:
class CustomLayer(keras.layers.Layer):
    def __init__(self, a):
        self.var = tf.Variable(a, name="var_a")

    def call(self, inputs, training=False):
        if training:
            return inputs * self.var
        else:
            return inputs

    def get_config(self):
        return {"a": self.var.numpy()}

    # 实际上，这里不需要定义from_config，因为返回cls（** config）是默认行为。
    @classmethod
    def from_config(cls, config):
        return cls(**config)


layer = CustomLayer(5)
layer.var.assign(2)

serialized_layer = keras.layers.serialize(layer)
new_layer = keras.layers.deserialize(
    serialized_layer, custom_objects={"CustomLayer": CustomLayer}
)

**注册自定义对象**

Keras记录了哪个类生成了配置。在上面的示例中，`tf.keras.layers.serialize`生成自定义层的序列化形式：

In [ ]:
{'class_name': 'CustomLayer', 'config': {'a': 2} }

Keras保留所有内置层，模型，优化器和指标类的主列表，该列表用于查找正确的类以调用`from_config`。如果找不到该类，则会引发错误（`Value Error: Unknown layer`）。有几种方法可以将自定义类注册到此列表中：

1. 在加载函数中设置`custom_objects`参数。（请参见上面“定义配置方法”部分中的示例）
2. `tf.keras.utils.custom_object_scope`或`tf.keras.utils.CustomObjectScope`
3. `tf.keras.utils.register_keras_serializable`

**自定义层和方法示例**

In [12]:
class CustomLayer(keras.layers.Layer):
    def __init__(self, units=32, **kwargs):
        super(CustomLayer, self).__init__(**kwargs)
        self.units = units

    def build(self, input_shape):
        self.w = self.add_weight(
            shape=(input_shape[-1], self.units),
            initializer="random_normal",
            trainable=True,
        )
        self.b = self.add_weight(
            shape=(self.units,), initializer="random_normal", trainable=True
        )

    def call(self, inputs):
        return tf.matmul(inputs, self.w) + self.b

    def get_config(self):
        config = super(CustomLayer, self).get_config()
        config.update({"units": self.units})
        return config


def custom_activation(x):
    return tf.nn.tanh(x) ** 2


# 使用CustomLayer和custom_activation建立模型
inputs = keras.Input((32,))
x = CustomLayer(32)(inputs)
outputs = keras.layers.Activation(custom_activation)(x)
model = keras.Model(inputs, outputs)

# 获取配置
config = model.get_config()

# 在加载时，使用`custom_object_scope`注册自定义对象：
custom_objects = {"CustomLayer": CustomLayer, "custom_activation": custom_activation}
with keras.utils.custom_object_scope(custom_objects):
    new_model = keras.Model.from_config(config)

#### 克隆内存中的模型

你还可以通过`tf.keras.models.clone_model()`克隆内存中模型。这等效于获取配置，然后从其配置中重新创建模型（因此它不保留编译信息或层权重值）。

**示例：**

In [ ]:
with keras.utils.custom_object_scope(custom_objects):
    new_model = keras.models.clone_model(model)

### 仅保存和加载模型的权重值

你可以选择仅保存和加载模型的权重。在以下情况下这可能很有用：

+ 你只需要模型进行预测：在这种情况下，你无需重新开始训练，因此你不需要编译信息或优化器状态。
+ 你正在进行迁移学习：在这种情况下，你将使用现有模型的状态来训练新模型，因此你不需要先前模型的编译信息。

#### **在内存中进行权重转移的API**

可以使用`get_weights`和`set_weights`在不同对象之间复制权重：

+ [`tf.keras.layers.Layer.get_weights()`](https://www.tensorflow.org/api_docs/python/tf/keras/layers/Layer?hl=zh_cn#get_weights)：返回numpy数组的列表。
+ [`tf.keras.layers.Layer.set_weights()`](https://www.tensorflow.org/api_docs/python/tf/keras/layers/Layer?hl=zh_cn#set_weights)：将模型权重设置为`weights`参数中的值。

在下面的示例中，**在内存中将权重从一个层转移到另一个层**

In [14]:
def create_layer():
    layer = keras.layers.Dense(64, activation="relu", name="dense_2")
    layer.build((None, 784))
    return layer


layer_1 = create_layer()
layer_2 = create_layer()

# 将权重从layer1复制到layer2
layer_2.set_weights(layer_1.get_weights())

在内存中，将权重从一个模型转移到具有兼容架构的另一个模型

In [15]:
# 创建一个简单的函数式模型
inputs = keras.Input(shape=(784,), name="digits")
x = keras.layers.Dense(64, activation="relu", name="dense_1")(inputs)
x = keras.layers.Dense(64, activation="relu", name="dense_2")(x)
outputs = keras.layers.Dense(10, name="predictions")(x)
functional_model = keras.Model(inputs=inputs, outputs=outputs, name="3_layer_mlp")

# 定义一个相同架构的子类化模型
class SubclassedModel(keras.Model):
    def __init__(self, output_dim, name=None):
        super(SubclassedModel, self).__init__(name=name)
        self.output_dim = output_dim
        self.dense_1 = keras.layers.Dense(64, activation="relu", name="dense_1")
        self.dense_2 = keras.layers.Dense(64, activation="relu", name="dense_2")
        self.dense_3 = keras.layers.Dense(output_dim, name="predictions")

    def call(self, inputs):
        x = self.dense_1(inputs)
        x = self.dense_2(x)
        x = self.dense_3(x)
        return x

    def get_config(self):
        return {"output_dim": self.output_dim, "name": self.name}


subclassed_model = SubclassedModel(10)
# 通过调用子类化模型创建权重
subclassed_model(tf.ones((1, 784)))

# 将函数式模型的权重复制给子类化模型
subclassed_model.set_weights(functional_model.get_weights())

assert len(functional_model.weights) == len(subclassed_model.weights)
for a, b in zip(functional_model.weights, subclassed_model.weights):
    np.testing.assert_allclose(a.numpy(), b.numpy())

**无状态层的情况**

由于无状态层不会更改权重的顺序或数量，因此即使存在额外的/缺少的无状态层，模型也可以具有兼容的架构。

In [16]:
inputs = keras.Input(shape=(784,), name="digits")
x = keras.layers.Dense(64, activation="relu", name="dense_1")(inputs)
x = keras.layers.Dense(64, activation="relu", name="dense_2")(x)
outputs = keras.layers.Dense(10, name="predictions")(x)
functional_model = keras.Model(inputs=inputs, outputs=outputs, name="3_layer_mlp")

inputs = keras.Input(shape=(784,), name="digits")
x = keras.layers.Dense(64, activation="relu", name="dense_1")(inputs)
x = keras.layers.Dense(64, activation="relu", name="dense_2")(x)

# 添加一个不包含权重的dropout层
x = keras.layers.Dropout(0.5)(x)
outputs = keras.layers.Dense(10, name="predictions")(x)
functional_model_with_dropout = keras.Model(
    inputs=inputs, outputs=outputs, name="3_layer_mlp"
)

functional_model_with_dropout.set_weights(functional_model.get_weights())

#### **用于将权重保存到磁盘并重新加载的API**

可以以下格式调用`model.save_weights`，将权重保存到磁盘：

+ TensorFlow Checkpoint
+ HDF5

`model.save_weights`的默认格式为TensorFlow checkpoint，有两种指定保存格式的方法：

1. `save_format`参数：将值设置为`save_format="tf"`或`save_format="h5"`。
2. `path`参数：如果路径以`.h5`或结尾`.hdf5`，则使用HDF5格式。除非`save_format`设置，否则其他后缀将保存为TensorFlow checkpoint。

还可以选择将权重作为内存中的numpy数组进行检索。每个API都有其优缺点，下面将详细介绍。

#### **TF Checkpoint格式**

**示例：**

In [17]:
# 可运行的模型
sequential_model = keras.Sequential(
    [
        keras.Input(shape=(784,), name="digits"),
        keras.layers.Dense(64, activation="relu", name="dense_1"),
        keras.layers.Dense(64, activation="relu", name="dense_2"),
        keras.layers.Dense(10, name="predictions"),
    ]
)
sequential_model.save_weights("ckpt")
load_status = sequential_model.load_weights("ckpt")

# `assert_consumed`可以用作验证是否已从checkpoint恢复所
# 有变量值。可以参阅`tf.train.Checkpoint.restore`以获取Status对象中的其他方法。
load_status.assert_consumed()

**格式细节**

TensorFlow Checkpoint格式使用对象属性名称保存和恢复权重。例如，思考该`tf.keras.layers.Dense`层。该层包含两个权重：`dense.kernel`和`dense.bias`。当该层被保存到`tf`格式，所得到的checkpoint 包含键`"kernel"`和`"bias"`以及它们相应的权重值。有关更多信息，请参见 [TF Checkpoint指南中的“加载机制”](https://www.tensorflow.org/guide/checkpoint?hl=zh_cn#loading_mechanics)。

请注意，属性/图是**以父对象中使用的名称而不是变量的名称**命名的。思考以下示例中的`CustomLayer`。变量`CustomLayer.var`将`"var"`作为key的一部分保存，`"var_a"`。

In [18]:
class CustomLayer(keras.layers.Layer):
    def __init__(self, a):
        self.var = tf.Variable(a, name="var_a")


layer = CustomLayer(5)
layer_ckpt = tf.train.Checkpoint(layer=layer).save("custom_layer")

ckpt_reader = tf.train.load_checkpoint(layer_ckpt)

ckpt_reader.get_variable_to_dtype_map()

{'_CHECKPOINTABLE_OBJECT_GRAPH': tf.string,
 'layer/var/.ATTRIBUTES/VARIABLE_VALUE': tf.int32,
 'save_counter/.ATTRIBUTES/VARIABLE_VALUE': tf.int64}

迁移学习的例子

本质上，只要两个模型具有相同的架构，它们就可以共享相同的checkpoint。

**示例：**

In [19]:
inputs = keras.Input(shape=(784,), name="digits")
x = keras.layers.Dense(64, activation="relu", name="dense_1")(inputs)
x = keras.layers.Dense(64, activation="relu", name="dense_2")(x)
outputs = keras.layers.Dense(10, name="predictions")(x)
functional_model = keras.Model(inputs=inputs, outputs=outputs, name="3_layer_mlp")

# 引用在“引入”部分中定义的一部分函数式模型。
# 以下几行代码产生一个新模型，该模型不包含函数式模型的最终输出层。
pretrained = keras.Model(
    functional_model.inputs, functional_model.layers[-1].input, name="pretrained_model"
)
# 随机分配"trained"权重
for w in pretrained.weights:
    w.assign(tf.random.normal(w.shape))
pretrained.save_weights("pretrained_ckpt")
pretrained.summary()

# 假设这是一个单独的项目，其中仅存在"pretrained_ckpt"。
# 使用一个不同的输出尺寸创建一个新的函数式模型
inputs = keras.Input(shape=(784,), name="digits")
x = keras.layers.Dense(64, activation="relu", name="dense_1")(inputs)
x = keras.layers.Dense(64, activation="relu", name="dense_2")(x)
outputs = keras.layers.Dense(5, name="predictions")(x)
model = keras.Model(inputs=inputs, outputs=outputs, name="new_model")

# 从pretrained_ckpt加载权重到模型中
model.load_weights("pretrained_ckpt")

# 检查所有的预训练权重是否被加载成功
for a, b in zip(pretrained.weights, model.weights):
    np.testing.assert_allclose(a.numpy(), b.numpy())

print("\n", "-" * 50)
model.summary()

# 示例2: Sequential模型
# 重新创建预训练模型，并加载权重
inputs = keras.Input(shape=(784,), name="digits")
x = keras.layers.Dense(64, activation="relu", name="dense_1")(inputs)
x = keras.layers.Dense(64, activation="relu", name="dense_2")(x)
pretrained_model = keras.Model(inputs=inputs, outputs=x, name="pretrained")

# Sequential例子:
model = keras.Sequential([pretrained_model, keras.layers.Dense(5, name="predictions")])
model.summary()

pretrained_model.load_weights("pretrained_ckpt")

# 警告! 调用`model.load_weights('pretrained_ckpt')`时出现错误
# 将无法按照预期工作，如果你检查权重，你将会发现权重没有被正常载入
# `pretrained_model.load_weights()`是正确的调用方法

Model: "pretrained_model"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
digits (InputLayer)          [(None, 784)]             0         
_________________________________________________________________
dense_1 (Dense)              (None, 64)                50240     
_________________________________________________________________
dense_2 (Dense)              (None, 64)                4160      
Total params: 54,400
Trainable params: 54,400
Non-trainable params: 0
_________________________________________________________________

 --------------------------------------------------
Model: "new_model"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
digits (InputLayer)          [(None, 784)]             0         
_________________________________________________________________
dense_1 (Dense)              (None, 64) 

通常建议使用相同的API构建模型。如果在“Sequential”和“函数式”或“函数式和子类化”等之间切换，则始终重建预训练模型并将预训练权重加载到该模型。

下一个问题是，如果模型架构完全不同，如何将权重保存并加载到不同的模型中？解决方案是使用[`tf.train.Checkpoint`](https://www.tensorflow.org/api_docs/python/tf/train/Checkpoint?hl=zh_cn)来保存和还原具体的层/变量。

**示例：**

In [20]:
# 创建一个只使用functional_model的第一层和最后一层的子类模型
# First, save the weights of functional_model's first and last dense layers.
first_dense = functional_model.layers[1]
last_dense = functional_model.layers[-1]
ckpt_path = tf.train.Checkpoint(
    dense=first_dense, kernel=last_dense.kernel, bias=last_dense.bias
).save("ckpt")

# 定义子类化模型
class ContrivedModel(keras.Model):
    def __init__(self):
        super(ContrivedModel, self).__init__()
        self.first_dense = keras.layers.Dense(64)
        self.kernel = self.add_variable("kernel", shape=(64, 10))
        self.bias = self.add_variable("bias", shape=(10,))

    def call(self, inputs):
        x = self.first_dense(inputs)
        return tf.matmul(x, self.kernel) + self.bias


model = ContrivedModel()
# 在输入上调用模型以创建密度层的变量
_ = model(tf.ones((1, 784)))

# 创建具有与以前相同结构的Checkpoint，然后加载权重。
tf.train.Checkpoint(
    dense=model.first_dense, kernel=model.kernel, bias=model.bias
).restore(ckpt_path).assert_consumed()

Instructions for updating:
Please use `layer.add_weight` method instead.


#### **HDF5格式**

HDF5格式包含按层名称分组的权重，权重是通过将可训练权重列表与不可训练权重列表（与`layer.weights`）连接起来并排序的列表。因此，如果模型具有与保存在checkpoint中相同的层和可训练状态，则可以使用hdf5的checkpoint。

示例：

In [21]:
# Runnable example
sequential_model = keras.Sequential(
    [
        keras.Input(shape=(784,), name="digits"),
        keras.layers.Dense(64, activation="relu", name="dense_1"),
        keras.layers.Dense(64, activation="relu", name="dense_2"),
        keras.layers.Dense(10, name="predictions"),
    ]
)
sequential_model.save_weights("weights.h5")
sequential_model.load_weights("weights.h5")

请注意，当模型包含嵌套层时，更改`layer.trainable`可能会导致`layer.weights`顺序不同 。

In [22]:
class NestedDenseLayer(keras.layers.Layer):
    def __init__(self, units, name=None):
        super(NestedDenseLayer, self).__init__(name=name)
        self.dense_1 = keras.layers.Dense(units, name="dense_1")
        self.dense_2 = keras.layers.Dense(units, name="dense_2")

    def call(self, inputs):
        return self.dense_2(self.dense_1(inputs))


nested_model = keras.Sequential([keras.Input((784,)), NestedDenseLayer(10, "nested")])
variable_names = [v.name for v in nested_model.weights]
print("variables: {}".format(variable_names))

print("\nChanging trainable status of one of the nested layers...")
nested_model.get_layer("nested").dense_1.trainable = False

variable_names_2 = [v.name for v in nested_model.weights]
print("\nvariables: {}".format(variable_names_2))
print("variable ordering changed:", variable_names != variable_names_2)

variables: ['nested/dense_1/kernel:0', 'nested/dense_1/bias:0', 'nested/dense_2/kernel:0', 'nested/dense_2/bias:0']

Changing trainable status of one of the nested layers...

variables: ['nested/dense_2/kernel:0', 'nested/dense_2/bias:0', 'nested/dense_1/kernel:0', 'nested/dense_1/bias:0']
variable ordering changed: True


#### **迁移学习的例子**

从HDF5加载预训练的权重时，建议将权重加载到原始checkpointed模型中，然后将所需的权重/层载入到新模型中。

**示例：**

In [23]:
def create_functional_model():
    inputs = keras.Input(shape=(784,), name="digits")
    x = keras.layers.Dense(64, activation="relu", name="dense_1")(inputs)
    x = keras.layers.Dense(64, activation="relu", name="dense_2")(x)
    outputs = keras.layers.Dense(10, name="predictions")(x)
    return keras.Model(inputs=inputs, outputs=outputs, name="3_layer_mlp")


functional_model = create_functional_model()
functional_model.save_weights("pretrained_weights.h5")

# In a separate program:
pretrained_model = create_functional_model()
pretrained_model.load_weights("pretrained_weights.h5")

# Create a new model by extracting layers from the original model:
extracted_layers = pretrained_model.layers[:-1]
extracted_layers.append(keras.layers.Dense(5, name="dense_3"))
model = keras.Sequential(extracted_layers)
model.summary()

Model: "sequential_6"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_1 (Dense)              (None, 64)                50240     
_________________________________________________________________
dense_2 (Dense)              (None, 64)                4160      
_________________________________________________________________
dense_3 (Dense)              (None, 5)                 325       
Total params: 54,725
Trainable params: 54,725
Non-trainable params: 0
_________________________________________________________________
